In [ ]:
## STANDARDIZZATORE ##

#FASI
#1. LETTURA estensione
#2. TRADUZIONE in JSON


#FORMATI DATASET (fonte: https://docs.italia.it/AgID/documenti-in-consultazione/lg-opendata-docs/it/bozza/allegato-b-standard-di-riferimento-e-formati-aperti.html)

#!! PRINCIPALI !!
##Formati aperti per i dati
#CSV (Comma Separated Values)
#JSON (JavaScript Object Notation)
#XML (eXtensible Markup Language)
#XLSX (Excel)
#!!


##Formati aperti più diffusi per i dati geografici
#Shapefile
#KML
#GeoJSON
#GML (Geography Markup Language)
#GeoPackage

##Formati aperti per i documenti
#ODF (Open Document Format)
#PDF
#Akoma Ntoso

##Formati per dati meteorologici
#BUFR (Binary Universal Form for the Representation of meteorological data)
#NetCDF (Network Common Data Form)
#ASCII (American Standard Code for Information Interchange)
#Avvisi Meteo: 
#CAP (Common Alerting Protocol), RSS (Really Simple Syndication)/Atom
#Radar: 
#HDF5 (Hierarchical Data Format)
#Modello NWP (Numerical weather prediction): 
#GRIB (General Representation of fields In Binary)


In [ ]:
import pandas as pd
import json
import os
import xml.etree.ElementTree as ET
from bson import ObjectId
import random

In [ ]:
def random_sampling(max_len, n_sample):
    random.seed(0)
    # Verifica che il numero di righe di cui fare sample non sia superiore al numero massimo effettivo di righe
    actual_n_sample = n_sample
    if (n_sample > max_len):
        actual_n_sample = max_len

    # Genera una lista di numeri casuali distinti nell'intervallo specificato
    row_list = random.sample(range(0, max_len), actual_n_sample)

    # Stampare la lista di numeri casuali distinti
    return row_list

In [ ]:
def extract(file_path, n_sample): 
    data = None
    nrow = 0
    root, extension = os.path.splitext(file_path)
    new_sample = None
    try:
        if (extension == '.csv'):
            data = pd.read_csv(file_path, encoding='latin1') 
            #conversione da effettuare perchè json ha problemi con colonne timestamp  
            new_sample = sample(data, n_sample)
            nrow = len(data)  
            #for col in new_sample.columns:
            #    if (pd.api.types.is_datetime64_any_dtype(new_sample[col])):
            #        new_sample[col] = new_sample[col].astype(str)
        elif (extension == '.xlsx'):
            data = pd.read_excel(file_path) 
            #conversione da effettuare perchè json ha problemi con colonne timestam                
            new_sample = sample(data, n_sample)  
            nrow = len(data)   
            #for col in new_sample.columns:
            #    if (pd.api.types.is_datetime64_any_dtype(new_sample[col])):
            #        new_sample[col] = new_sample[col].astype(str)
    except Exception as e: #se ha problemi a leggere file tabulari sbagliati
        data = None
        # Ignora l'eccezione e continua senza stampare l'errore
        pass

    
    
    return new_sample, nrow
        
def sample(data, n_sample):
    row_sampled = random_sampling(len(data),n_sample)
    sample_data = data.loc[row_sampled]
    return sample_data
    
#def extract_json(file_path, n_sample):
#    with open(file_path, 'r') as file:
#        data = json.load(file)
#    return data




In [ ]:
#mi verifica se la lista è numerica nonostante la presenza di stringhe
def is_numeric_variable(lst):
    try:
        # Prova a convertire ogni elemento in float
        float_values = [float(element) for element in lst]
        return True
    except Exception as e:
        # Se la conversione in float genera un errore, la lista non è completamente numerica
        return False



#identifico il tipo di variabile fra:
#numerica discreta
#numerica continua
#categorica (parole per le categorie) o testuale (solo testi diversi)?

def variable_type_detector(variable):
    vtype = ""
    #è numerica (tipo int, o stringhe che rappresentano numeri)
    if(pd.api.types.is_numeric_dtype(variable) | is_numeric_variable(variable)):
        #conta ripetizioni di valori
        unique_counter = variable.nunique()
        #al massimo 15 categorie (esempio)
        max_counter = 15 
        if (unique_counter <= max_counter):
            vtype = "categorical numerical"
        else:
            vtype = "continuous numerical"
    #è testuale
    else:
        #conta ripetizioni di valori
        unique_counter = variable.nunique()
        #al massimo 15 categorie (esempio)
        max_counter = 15 
        if (unique_counter <= max_counter):
            vtype = "categorical textual"
        else:
            vtype = "text"
            
    return vtype
        

In [ ]:
def create_file_node(file_name, sample_data, nrow):
    #default values
    file_elements = file_name.split(".")
    extension_type = "undefined"
    features = []
    n_features = 0
    
    if (file_elements[1] in ['csv', 'xlsx']):
        extension_type = "tabular"
        n_features = len(sample_data.columns)
        #controlla se è time stamp la feature, la converte in stringa, altrimenti json non va
          
        
        features = [
        {
            "feature_name": col,
            "feature_datatype": sample_data[col].dtype.name,
            "feature_type": variable_type_detector(sample_data[col]),
            "elements_sampled": sample_data[col].tolist()
        }
        for col in sample_data.columns         
            
    ]   
    elif (file_elements[1] in ['jpg', 'png']):
        extension_type = "image"   
    
    
    new_file = {
        "file_name": file_elements[0],
        "file_extension": file_elements[1],
        "file_type": extension_type,
        "n_istances": nrow,
        "n_features": n_features,
        "features": features
    }
    return new_file

In [ ]:
def navigate_folders(root, subfolders_limit, files_limit, new_dataset_node, subfolder_count=0):
#mi prende eventuali files, i primi 10
    for current_root, dirs, files in os.walk(root):
        for file in files[0:files_limit]:
            file_path = os.path.join(current_root, file)
            n_sample = 100
            result_sample = extract(file_path, n_sample)
            file_sample = result_sample[0]
            nrow = result_sample[1]
            if file_sample is not None:
                new_file = create_file_node(file, file_sample, nrow)
                new_dataset_node["files"].append(new_file)
                #print(file)
                
                
        #se c'è una sottocartella, "scava" ad albero, limitando il numero di sottocartelle da esaminare
        # Chiamata ricorsiva per le prime massime sottocartelle
        if subfolder_count >= subfolders_limit:
            break
            
        for subfolder in dirs:
            new_root = os.path.join(current_root, subfolder)
            subfolder_count = subfolder_count + 1
            navigate_folders(new_root, subfolders_limit, files_limit, new_dataset_node, subfolder_count)
            # Verifica se abbiamo raggiunto il limite delle sottocartelle
            if subfolder_count >= subfolders_limit:
                break
            

In [ ]:
#Navigation

#Directory
root_directory = 'kaggle_datasets'
output_root_directory = 'datapoints'
extensions = ['.csv', '.json', '.xlsx']
directories = [dirc for dirc in os.listdir(root_directory)]
#files = [file for file in os.listdir(directory) if file.endswith(tuple(extensions))]

# Json final
datasets_json = {}
# Converte ciascun file in un JSON
for directory in directories:
    category = directory 
    directory_path = root_directory + "/" + directory
    #each folder is the name of the dataset
    datasets = [dataset for dataset in os.listdir(directory_path)]
    #in ciascuna cartella del dataset, posso aspettarmi di tutto, cartelle, files etc.. come faccio?
    #possibile soluzione: prendo alpiù 5 cartelle e alpiù 10 files, nelle cartelle scavo ad albero in 1-2 livelli massimi,
    #poi mi fermo e inserisco max 10 files nella struttura
    for dataset in datasets:
        dataset_path = root_directory + "/" + directory + "/" + dataset
        new_dataset_node = {"_id": dataset, "label": category, "files": []}
        navigate_folders(dataset_path, 5, 10, new_dataset_node)
        
        json_data = json.dumps(new_dataset_node, indent=2, default=str)
        json_path = output_root_directory  + "/" + directory + "/"
        with open( json_path + dataset + ".json", "w") as json_file:          
            json_file.write(json_data)
        

In [ ]:
#tests..

In [ ]:
# Carica il file CSV
file_path = 'kaggle_datasets/clustering/customer-segmentation-dataset/Online Retail.xlsx'
df = pd.read_excel(file_path)

# Ottieni i metadati
info = df.info()

# Visualizza i metadati
print(info)